In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.kears import backend as K
import matplotlib.pyplot as plt
from Sklearn.model_selection import train_test_split

ImportError: cannot import name 'learningRateScheduler' from 'tensorflow.keras.callbacks' (c:\Users\dell g15\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\api\_v2\keras\callbacks\__init__.py)